**1. (i) In this simple assignment, you will implement a single layered perceptron algorithm. You will need to write in a language of your choice
from SCRATCH (use of pre-built machine learning libraries is not allowed though you can use basic linear algebra routines from numpy
and scipy). (ii) Then use the MNIST dataset to find the accuracy that
a single layered perceptron that you have implemented gives. Note the
MNIST dataset requires classification into one of 10 categories (digits). You may assume that the class indicated is given by the output
of the corresponding neuron with the highest output. Use 5-fold cross
validation**

In [364]:

#Importing Libraries
import sklearn
import numpy as np
from sklearn import datasets
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split

In [365]:
def InitializeWeights(n,m): # Where n is the no of input features and m is the no of Classes
  W = np.random.randn(m, n)
  return W

In [366]:
def Activation(h):
  result = 1 / ( 1 + np.exp(-h))
  return result

def ActivationDerivative(h):
  result = Activation(h)*(1 - Activation(h))
  return result


In [367]:
def ForwardPass(sample,W):
  m = len(W)
  predicted = [0]*m
  h = [0]*m
  for i in range(0,m):
      h[i] = W[i].T @ sample
      predicted[i] = Activation(h[i])
  return h,predicted


In [368]:
def UpdateWeights(sample,m,n,learning_rate,label,W,del_W):
  # To update weights based on the gradients
  h,predicted = ForwardPass(sample,W)
  for i in range(0,m):
    for j in range(0,n):
      J_W = 2 * (predicted[i]-label[i]) * sample[j]
      del_W[i][j] = del_W[i][j]  - (learning_rate * J_W)
  return del_W


In [369]:
def SingleLayerPerceptron(train,label,learning_rate,epochs):
  n = len(train[0]) # No.of.input features
  m = len(label[0]) # No.of classes
  
  W = InitializeWeights(n,m) #Initialsing weights to random values
  
  for i in range(0,epochs):
    del_W = np.zeros((m, n))
    for j in range(0,len(train)):
      del_W = UpdateWeights(train[j],m,n,learning_rate,label[j],W,del_W)
    W = W + del_W
  return W
  

In [370]:
#loading MNIST dataset

mnist = datasets.load_digits()
m_data = mnist.data

# One-hot encoding of the target
y = []
for i in mnist.target:
  L = [1  if i == j else 0 for j in range(10)]
  y.append(L)

# Adding 1 at the beginning of each input array for bias
X =[]
for i in range(len(m_data)):
  X.append(np.concatenate([[1],m_data[i]]))

In [371]:
# To calculate the accuracy of the model
def CalculateScore(test,label,W_predict):
  sum = 0
  #CM =[[0]*10 for _ in range(10)] : To print confusion matrix if required
  for i in range(len(test)):
    a,b = ForwardPass(test[i],W_predict)
    max_prob = max(b)
    predicted_value = b.index(max_prob)
    #CM[label[i].index(1)][predicted_value] += 1 : To print confusion matrix if required
    if label[i].index(1) == predicted_value:
      sum = sum + 1
  #print("CONFUSION MATRIX : \n",np.matrix(CM)) : To print confusion matrix if required
  return sum/len(test) 

In [372]:
# Finding accuracy of the model using 5-fold Cross Validation
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=0) 
score = 0
for train_index, test_index in sss.split(X, y):
    X_train = [];X_test = [];y_train =[];y_test =[]
    print("Processing a split of Train and Test data")
    for i in train_index:
      X_train.append(X[i])
      y_train.append(y[i])
    for j in test_index:
      X_test.append(X[j])
      y_test.append(y[j])
    W_predict = SingleLayerPerceptron(X_train,y_train,0.1,10)
    s = CalculateScore(X_test,y_test,W_predict)
    print("Score:",s)
    score += s
print("Average Accuracy:",(score/5)*100,"%")

Processing a split of Train and Test data


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


Score: 0.7388888888888889
Processing a split of Train and Test data
Score: 0.7694444444444445
Processing a split of Train and Test data
Score: 0.6861111111111111
Processing a split of Train and Test data
Score: 0.7305555555555555
Processing a split of Train and Test data
Score: 0.7305555555555555
Average Accuracy: 73.11111111111111 %


**2. For this part you can use scikit (or any other pre-packaged library)
and use their built-in implementation of multi-layered perceptron. Use
the same dataset as above. Use 5-fold cross validation as above and
compare your results with what you get with a single layered perceptron**

In [373]:
mnist = datasets.load_digits()
X = mnist.data
y = mnist.target

In [378]:
# Classification using the sklearn MLP classifier

clf = MLPClassifier(hidden_layer_sizes = (100),activation='logistic',learning_rate='constant', learning_rate_init=0.001, max_iter=3000)
L = cross_val_score(clf, X, y, cv=5)
print("Average accuracy with a Multilayered Perceptron:",sum(L)/5)

# Cross-Validaton using Stratified Split

score = 0
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=0) 
for train_index, test_index in sss.split(X, y):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  clf.fit(X_train,y_train)
  score += clf.score(X_test,y_test)
print("Average accuracy with a Multilayered Perceptron (using stratified split):" , (score*100)/5,"%")

Average accuracy with a Multilayered Perceptron: 0.9432466728567007
Average accuracy with a Multilayered Perceptron (using stratified split): 97.61111111111111 %


**From the above models, we can see that the accuracy obtained using a Multilayer perceptron is  significantly higher than the accuracy obtained using a Single Layer perceptron.** 
